# 개요

- [Attention is all you need](https://arxiv.org/abs/1706.03762)
    - 2017년 구글이 제안한 새로운 신경망 아키텍처(트렌스포머)를 소개한 논문
    - 트랜스포머는 RNN을 능가 함
- [ULMFit: Universal Language Model Fine-tuning for Text Classification](https://arxiv.org/abs/1801.06146)
    - 효율적인 전이학습    
    - 매우 큰 말뭉치에서 LSTM을 사용, 적은 양의 레이블로도 최고 수준의 텍스트 분류 모델 생성 가능 입증
- GPT(Generative Pretrained Transformer)과 BERT(Bidirectional Encoder Representations form Transformer)
    - 트렌스포머 아키텍처와 비지도학습을 결합
    - 작업에 특화된 모델을 밑바닥부터 훈련 시킬 필요 없앰
    - 거의 모든 NLP 벤치마크에서 큰 차이로 기록 갱신
    
*그림 1-1 트렌스포머 모델의 타임라인*
<img src="./nlp-with-transformers-main/images/chapter01_timeline.png" width=500>

# 인코더-디코더 프레임워크

*그림 1-2 : 시간의 축을 따라 펼친 RNN*
<img src="./nlp-with-transformers-main/images/chapter01_rnn.png" width=500>

- RNN 시대
    - 트랜스포머 이전은 LSTM과 같은 순환 신경망 구조가 NLP에서 최고 수준의 성능 달성
    - 순환신경망 구조는, 정보를 한 스텝에서 다음 스텝으로 전파 하도록 네트워크에 피드백 루프 포함
    - 그림 1-2의 왼쪽을 보면, RNN은 (단어 또는 문자와 같은) 입력을 받아 네트워크를 통과 하여 은닉 상태 라는 벡터 출력
    - 출력된 정보를 동시에 피드백 루프로 보내 자기 자신에게도 입력
    - 그림 1-2의 오른쪽은, 피드벡 루프를 시간 축에 따라 펼친 것 도식화 

- seq2seq
    - RNN은 인코더-디코더 구조로 단어 시퀀스를 한 언어에서 다른 언어로 매핑하는 기계 번역에 사용
    - 입력과 출력이 임의의 길이를 가진 시퀀스에 적절
    - 인코더는 입력 시퀀스 정보를, **마지막 은닉 상태**라고 부르는 수치 표현으로 인코딩 하여 디코더에 전달 
    - 인코더의 마지막 은닉 상태가 **정보 병목(information bottlenec)** 이 된다는 약점
    - 시퀀스가 긴 경우 모든 것을 고정된 하나의 표현으로 압축 하는 과정에서 입력 시퀀스 초반 정보가 손실될 가능성
    - 어텐션으로 모든 인코더의 은닉 정보에 접근 하게 하여 이 문제 해결
    
*그림 1-3 한 쌍의 RNN으로 구성된 인코더-디코더 구조*
<img src="./nlp-with-transformers-main/images/chapter01_enc-dec.png" width=500>

# 어텐션 메커니즘

- 어텐션 기본 개념
    - 입력 시퀀스에서 은닉 상태를 만들지 않고
    - 스텝마다 인코더에서 디코더가 참고할 은닉 상태를 출력 한다
    - 모든 상태를 동시에 사용하려면 디코더에 많은 입력 발생
    - 어떤 상태를 먼저 사용할지 우선순위 결정 메커니즘 필요 --> 이것이 어텐션
    - 디코더가 모든 디코딩 타임스텝(timestep)마다 인코더의 각 상태에 다른 가중치 ('어텐션')을 할당
    - 그림 1-4에, 어텐션은 출력 시퀀스에 있는 두 번째 토큰을 예측 하는 역할
*그림 1-4 한 쌍의 RNN과 어텐션 메커니즘으로 구성된 인코더-디코더 구조*
<img src="./nlp-with-transformers-main/images/chapter01_enc-dec-attn.png" width=500>

- 사례로 보는 어텐션 작동 원리
    - 어텐션 기반 모델은 타임스텝마다 가장 많이 관련된 입력 토큰에 초점을 맞춤
    - 번역에 활용 가능할 경우, 원 문장과 번역할 문장의 각 단어의 복잡한 상관 관계를 학습
    - 그림 1-5는 영어-프랑스어 번역 모델의 어텐션 가중치를 시각화 한 것
    - 여기서 각 픽셀은 각각의 가중치를 의미 
    - 'zone'과 'Area'를 디코더가 어떻게 올바르게 정렬하는지 보여 줌

*그림 1-5 RNN 인코더-디코더에서 영어 단어와 프랑스어 번역 단어의 정렬*
<img src="./nlp-with-transformers-main/images/chapter01_eng_fr.png" width=300>

- 패러다임을 바꾼 트렌스포머
    - 어텐션으로 번역 성능이 올라 갔지만, 여전히 RNN을 사용한 단점 존재
    - 계산이 순차적이라, 병렬화 어려움
    - RNN 순환을 제거하고 셀프 어텐션(self-attention) 도입
    - 신경망의 같은 층에 있는 모든 상태에 대해 어텐션 적용
    - 그림 1-6은 인코더-디코더에 셀프 어텐션을 활용한 구조 도식화
    - 어텐션 출력은 fead-forward neural network(FF FN)에 주입
    - 순환 모델 보다 훨씬 더 바르게 훈련

*그림 1-6 원본 트랜스포머의 인코더-디코더 구조*
<img src="./nlp-with-transformers-main/images/chapter01_self-attention.png" width=500>

# NLP의 전이학습

- 전이학습의 필요성
    - NLP 어플리케이션이 모델 훈련에 사용할 대규모 텍스트 데이터는 구하기 어렵다
    - 전이 학습을 통해 이 문제를 해결 가능
    - 전이 학습 이란, 다양한 작업에서 적은 양의 레이블 데이터로 훨씬 효과적으로 훈련한 높은 품질의 모델을 만드는 것

- 전의학습의 원리
    - 모델은 body와 header 로 구성 (그림 1-7(오른쪽)
    - body의 가중치는 훈련하는 동안 원래의 domain의 다양한 특성 학습
    - body의 가중치는 새로운 모델을 위한 초기값으로 사용
    - 도메인을 옮겨 적은 수의 레이블 데이터로 학습 진행
    - 컴퓨터 비전 분야에서는 높은 성능을 내며 사용 됨

*그림 1-7 전통적인 지도 학습(왼쪽)과 전이 학습(오른쪽)의 비교*
<img src="./nlp-with-transformers-main/images/chapter01_transfer-learning.png" width=500>

- NLP 분야의 전이학습 기술 발달
    - 컴퓨터 비전 분야와 다르게 NLP 분야에서는 전이 학습의 성능이 낮음
    - OpenAI 연구원들이 감성 분류 작업에 비지도 사전 훈련에서 추출한 특성을 사용해 높은 성능
        - [Learning to Generate Reviews and Discovering Sentiment](https://arxiv.org/abs/1704.01444)
    - 범용적인 프레임워크 [ULMFiT](https://arxiv.org/abs/1801.06146)가 등장
        - 다양한 작업에 사전 훈련된 LSTM 모델 사용

- ULMFiT 프로세스
    - 그림 1-8과 같이 세 개의 주요 단계로 구성
    - **1. 사전 훈련**
        - 목표 : 이전 단어를 바탕으로 다음 단어를 예측
        - 이를 언어 모델링(language modeling) 이라고 함
        - 레이블링된 데이터 불필요 (위키피디아 같은 풍부한 소스 활용)
    - **2. 도메인 적용**
        - 사전 훈련이 끝난 모델을, 도메인 내 말뭉치에 적응 
        - 여전히 동일한 학습(이전 단어로 다음 단어 예측) 방법 이지만 target domain 의 말뭉치 사용
    - **3. 미세튜닝**
        - 분류 layer 를 사용하여 미세 튜닝 (그림 1-8의 경우 영화 리뷰 감성 분류)

*그림 1-8 ULMFiT 프로세스*
<img src="./nlp-with-transformers-main/images/chapter01_ulmfit.png" width=500>


- **GPT**
    - 트랜스포머 아키텍처의 디코더 부분만 사용
    - ULMFiT 같은 언어 모델링 방법 사용
    - [BookCorpus](https://arxiv.org/abs/1506.06724) 데이터셋 으로 사전 훈련 (7천권의 다양한 장르의 도서)

- **BERT**
    - 트랜스포머 아키텍처의 인코더 부분만 사용
    - masked language modeling 이라는 특별한 형태의 언어 모델링 사용
        - 텍스트에서 랜덤하게 masking 된 단어를 예측 하는 것
        - 'I looked at my [MASK] and saw that [MASK] was late' 라는 문장에 [MASK] 부분의 단어 예측
    - BookCorpus과 위키피디아를 사전 훈련에 사용

- 허깅페이스의 등장
    - 연구실마다 프레임워크(파이토치, 텐서플로우)를 사용해 서로 호환되지 않는 모델을 릴리즈
    - NLP 연구자들이 이런 모델을 사용하기 곤란
    - [허깅페이스 트랜스포머스 : Hugging Face Transformers](https://huggingface.co/docs/transformers/index)의 등장
    - 단일화된 API 구축 시작

# 허깅페이스 트렌스포머스

- 새로운 머신러닝 아키텍처를 새로운 작업에 적용하는 단계
    - 1. 모델 아키텍처를 코드로 구현, 파이토치, 텐서플로우 사용
    - 2. 서버로 부터 사전 훈련된 가중치 다운로드
    - 3. 입력을 전처리 하고 모델에 전달, 및 사후 처리
    - 4. 데이터로더를 구현, 모델 훈련을 위한 손실 함수와 옵티마이저 정의
    
- 문제점
    - 모델과 작업 내용 별 별도의 사용자 정의 로직 필요
    - 공개 되는 코드의 표준화가 되어 있지 않아 수정에 많은 시간 소요

- 허깅페이스 트랜스포머스의 특장점
    - 매우 다양한 트랜스포머 모델에 표준화된 인터페이스 제공
    - 새로운 문제에 모델을 적용하는 코드와 도구 제공
        - 파이토치, 텐서플로우, [JAX](https://modulabs.co.kr/blog/what-is-jax-flax/) 대응
    - 작업에 맞는 head를 제공하여 미세튜닝 (분류, 개체명 인식, 질의 응답 등) 용이
    - 개발 및 훈련, 테스트 등에 시간 단축

# 트랜스포머 애플리케이션 둘러보기
- 감정분석 
- 온라인 주문에 대한 가짜 피드백 예제
    - 한국어 번역 : 
    > 친애하는 Amazon, 지난 주에 저는 독일에 있는 귀하의 온라인 상점에서 Optimus Prime 액션 피규어를 주문했습니다. 불행하게도, 패키지를 열었을 때 나는 그 대신 메가트론의 액션 피규어를 받았다는 사실을 알고 경악했습니다! 디셉티콘의 평생적인 적으로서 내 딜레마를 이해해 주시기 바랍니다. 이 문제를 해결하기 위해 제가 주문한 옵티머스 프라임 피규어를 메가트론으로 교환해 달라고 요청합니다. 이 구매에 관한 내 기록의 사본이 동봉되어 있습니다. 곧 연락을 기다리겠습니다. 감사합니다, 범블비.
    

In [5]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the ackage, I discovered to my horror that I had been sent an action figure of Megatron instead! 
As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. 
I expect to hear from you soon. Sincerely, Bumblebee."""

## 텍스트 분류
- 파이프라인 사용 모델 다운 로드 및 파인 튜닝
    - 원시 텍스트를 미세 튜닝된 모델의 예측으로 변환하기 위한 모든 단계를 추상화
    - pipeline() 함수를 호출 하며, 관심이 있는 작업 이름을 전달해 파이프라인 겍체 생성
    - 자동으로 [허깅페이스 허브](https://huggingface.co/models) 에서 모델 가중치를 다운 로드
    - 한번 다운 받으면, 캐싱된 버전을 사용 함으로 다운 로드를 생략
    - 'text-classification' 파이프라인은 감성 분석, 다중 분류, 다중 레이블 분류 지원

In [6]:
from transformers import pipeline

classifier = pipeline('text-classification')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


- 예측 해 보기
    - 각 파이프라인은 텍스트 문자열(또는 문자열의 리스트)를 입력으로
    - 예측 리스트를 출력으로 반환
    - 각 예측은 하나의 파이썬 딕셔너리로 pandas 로 출력 가능
    - 아래의 결과는, 텍스트가 부정적으로 확신

In [7]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.928078


## 개체명 인식
- 개체명(named entity)
    - NLP에서 제품, 장소, 사람 같은 실제 객체를 지칭
    - 개체명 인식(NER, named entity recognition) : 개체명을 텍스트에서 추출 하는 과정
    - aggregation_strategy 로 단어를 그룹화
    - ORG(조직), LOC(위치), PER(사람) 과 같은 카테고리에 할당
    - score는 얼마나 확신을 하는가
    - word 열의 ##은 토크나이저가 생성 

In [8]:
ner_tagger = pipeline('ner', aggregation_strategy ='simple')
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

,entity_group,score,word,start,end
0,ORG,0.856558,Amazon,5,11
1,MISC,0.989654,Optimus Prime,36,49
2,LOC,0.999747,Germany,90,97
3,MISC,0.533730,Mega,208,212
4,PER,0.568252,##tron,212,216
5,ORG,0.544610,Decept,254,260
6,MISC,0.543417,##icons,260,265
7,MISC,0.759125,Megatron,352,360
8,MISC,0.986541,Optimus Prime,369,382
9,PER,0.813238,Bumblebee,506,515


## 질문 답변
- 텍스트 구절과 함께 답을 얻고 싶은 질문을 모델에 전달
- 모델은 답변 텍스트를 변환
- 추출적 질문 답변 : 답변을 텍스트에서 직접 추출 

In [10]:
reader = pipeline('question-answering')
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.579689,337,360,an exchange of Megatron


## 요약
- 긴 텍스트를 입력으로 받아 관련 사실이 모두 포함된 간단한 버전 생성

In [11]:
summarizer = pipeline('summarization')
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Your min_length=56 must be inferior than your max_length=45.
/home/restful3/anaconda3/envs/trading/lib/python3.10/site-packages/transformers/generation/utils.py:1380: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (45). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. When he opened the ackage, he discovered to his horror that he had been sent an action figure of Megatron instead.


## 번역

In [17]:
translator = pipeline("translation_en_to_de", 
                     model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur von Ihrem Online-Shop in Deutschland bestellt. Leider, als ich den Angriff öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, hoffe ich, dass Sie mein Dilemma verstehen können. Um das Problem zu lösen, fordere ich einen Austausch von Megatron für die Optimus Prime Figur, die ich bestellt hatte. Eingeschlossen sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von Ihnen zu hören. Aufrichtig, Bumblebee.


In [26]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-ko")
outputs = translator('Bumblebee ordered an Optimus Prime action figure from your online store in Germany.')
print(outputs)

[{'translation_text': '모든 숫자는 완전히  역사적인885 NORETH 창조자 방콕 에 따라.'}]


In [25]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ko-en")
outputs = translator('안녕.')
print(outputs)

[{'translation_text': 'Hey.'}]


## 텍스트 생성
- 자동 완성 기능으로 고객 피드벡에 빠르게 응답 하는 시나리오 가정

In [27]:
generator = pipeline("text-generation")
response = "Dear Bumplebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the ackage, I discovered to my horror that I had been sent an action figure of Megatron instead! 
As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. 
I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumplebee, I am sorry to hear that your order was mixed up. 

I am so sorry that I asked you to please exchange your order because your account went through a major processing glitch when I asked you if you would like your current Optimus Prime figure purchased today. 

I do not feel compelled to receive your order.


# 허깅페이스 생태계

- 허깅페이스 트렌스포머스는 NLP와 머신러닝 프로젝트의 속도를 높이는 다양한 라이브러리와 도구를 갖춘 생태계로 빠르게 성장 (그림 1-9)
- 라이브러리와 허브로 구성
    - 라이브러리는 코드 제공
    - 허브는 사전에 훈련된 모델 가중치, 데이터셋, 평가 지표를 위한 스크립트 제공
        
<img src="./nlp-with-transformers-main/images/chapter01_hf-ecosystem.png" width=500>


## 허깅페이스 허브

- 전이학습은 트렌스포머 성공의 주요 원인 중 하나
    - 사전 훈련된 모델을 빠르게 로드/테스트 가능
- 무료로 공개된 20,000여 개의 모델 호스팅(그림 1-10)
- 작업, 프레임워크, 데이터셋 등으로 필터링 하여 모델 검색
- 모델 로드는 코드 한줄로 가능


*그림 1-10 허깅페이스 허브의 모델 페이지, 왼쪽에 필터, 오른쪽에 모델 목록*

<img src="./nlp-with-transformers-main/images/chapter01_hub-overview.png" width=500>


- 허브는 모델 가중치 외에 데이터셋과 평가 지표 계산을 위한 스크립트 제공
- 공개된 결과를 재현하거나 어플리케이션에 활용 가능한 추가 데이터 획득에 용이
- 모델 카드와 데이터셋 카드 : 모델과 데이터셋 내용을 문서화 (그림 1-11)
- 파이토치와 텐서플로우 각각 허브를 제공

*그림 1-11 허깅 페이스 허브에 있는 모델 카드, 오른쪽에 있는 추론 위젲에서 모델을 실험 가능*
<img src="./nlp-with-transformers-main/images/chapter01_hub-model-card.png" width=500>


## 허깅페이스 토크나이저
- 토큰화 단계 : 텍스트를 토큰이라는 더 작은 단위로 나누는 단계
- 단어, 단어의 일부, 구두점 같은 문자도 토큰이 된다
- 트랜스포머 모델은 토큰 단위로 훈련
- 허깅페이스의 토크나이저는 다양한 토큰화 전략 제공
- [Rust](https://www.rust-lang.org/) backend 덕분에 빠른 텍스트 토큰화 제공
- 입력 정규화, 모델 출력을 적절한 포맷으로 변환 등 사후 처리 제공
- 허깅페이스 트렌스포머로 사전 훈련된 모델을 로딩 하듯 토크나이저 로딩 후 사용

## 허깅페이스 데이터셋

- 모델을 훈련하고 평가에 사용
- 허깅페이스 데이터셋은 허브에서 데이터셋을 찾는 표준 인터페이스 제공
- 메모리 패핑 : 스마트한 캐싱을 제공하여 램 부족 회피
    - 파일 내용을 가상의 메모리에 저장, 여러게의 프로세스로 더 효율적으로 파일 수정
    - pandas, numpy 등과 상호 운영이 가능하여 데이터 랭글링 도구로도 활용 가능
- 많은 평가 지표를 위한 스크립트 제공

## 허깅페이스 엑셀러레이트
- 사용자 정의 로직을 처리하는 일반적인 훈련 루프에 훈련 인프라에 필요한 추상화 층 추가
- 인프라 전환을 단순화해 워크플로우 가속화
- 개인이 작성한 파이토치 훈련 스크립트를 회사 클러스터에 포팅 하는 등에 활용
- [🤗 Accelerate](https://huggingface.co/docs/accelerate/index)
    - 🤗 Accelerate는 단 4줄의 코드만 추가하여 모든 분산 구성에서 동일한 PyTorch 코드를 실행할 수 있게 해주는 라이브러리입니다! 
    - 대규모 교육과 추론을 통해 간단하고 효율적이며 적응력이 높아졌습니다.

# 트랜스포머의 주요 도전 과제

- **언어**
    - NLP에는 거의 영어 사용
    - 다중 언어 트랜스포머와 제로샷 교차언어 전이 수행은 4장에서

- **데이터 가용성**
    - 모델에 필요한 레이블링된 훈련 데이터의 양은 전이 학습을 사용하며 크게 줄었지만,
    - 레이블링에 들어가는 수고로움을 생각하면 여전히 많다.
    - 레이블링된 데이터가 없거나 부족한 상황에 대한 대처 방법은 9장에서

- **긴 문서 처리하기**
    - 셀프 어텐션은 텍스트 길이가 문단 정도 될 때 잘 작동
    - 문서와 같이 긴 길이의 텍스트 사용시 고 비용 발생
    - 이에 대한 대처 방법은 11장 에서

- **불투명성**
    - 다른 딥러닝 모델에 비해 불투명한 구조 임
    - 모델의 예측한 결과를 설명하기가 쉽지 않음
    - 중요한 결정에 모델 사용시 문제가 될 수 있음
    - 2, 4 장에서 트렌스포머 모델의 오류를 조사하는 방법

- **편향**
    - 트렌스포머 모델은 인터넷 텍스트를 사용하여 사전 훈련
    - 이런 데이터의 편향이 모델에 그대로 반영
    - 인종차별, 성차별 등, 10장에서 관련 이슈 

# 결론
- 분류, 개체명 인식, 질문 대답, 번역, 요약 을 위한 최고 수준의 모델을 단 몇줄의 코드로 사용
- 이어지는 장에서, 텍스트 분류 모델, 제품을 위한 경량 모델 구축, 언어 모델을 처음 부터 훈련 등 배움
- 이 책은 실습 위주로, GPU 로컬 머신에서 구동 가능